In [1]:
!python --version

Python 3.13.7


In [2]:
import json

with open('23092025_graph.json', mode='r', encoding='utf-8') as f:
    data = json.load(f)

list(data.keys())

['Nvidia', 'AMD', 'Intel']

In [3]:
from google import genai
from dotenv import load_dotenv
import os

load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY", "")
client = genai.Client(api_key=GEMINI_API_KEY)

In [19]:
summary7day_prompt = """
You are an analyst specializing in the semiconductor industry. 

TASK:
Summarize risks and opportunities for **{chipmaker}** over the last 7 days based on extracted knowledge triplets.

INPUT FORMAT:
Each triplet has:
- subject
- relation (verb + optional detail)
- object

Triplets (past 7 days):
{triplets}

INSTRUCTIONS:
1. Carefully analyze the triplets to identify signals of **risks** (e.g., export restrictions, regulatory scrutiny, market struggles, competition, delays).
2. Identify **opportunities** (e.g., new product launches, strong demand, partnerships, positive analyst ratings, technological advances).
3. For each risk or opportunity, **cite the supporting triplet** in this format:
   subject -(verb[/detail])-> object
   Example: China -(scrutinizes)-> Nvidia
4. Organize into two sections:
   - **Risks for {chipmaker}**
   - **Opportunities for {chipmaker}**
5. At the end, write a **short summary outlook** combining both risks and opportunities.

OUTPUT FORMAT:
Risks for {chipmaker}:
- [triplet] → explanation
- ...

Opportunities for {chipmaker}:
- [triplet] → explanation
- ...

Outlook:
<short paragraph>
"""

In [20]:
chipmaker = "Nvidia"
triplets = [t for d in data[chipmaker] for t in d['triplets']]

response = client.models.generate_content(
    model="gemini-2.5-flash", 
    contents=summary7day_prompt.format(
        chipmaker=chipmaker, 
        triplets=triplets
    )
)

print(response.text)

As an analyst specializing in the semiconductor industry, here's a summary of risks and opportunities for Nvidia over the last 7 days:

### Risks for Nvidia:
- **Geopolitical and Market Access Restrictions:**
    - Beijing -(bans/from using Nvidia microchips)-> companies
    - China -(bans/Nvidia AI)-> chips
    - China -(scrutinizes)-> Nvidia
    - Huang -(states/on US China AI dominance)-> patience
    - Nvidia -(exports/to China)-> chips
    → Mounting bans from Beijing and China on the use of Nvidia's microchips and AI chips by local companies, coupled with broader scrutiny, represent a significant geopolitical risk. These restrictions impact market access and export volumes, as highlighted by CEO Jensen Huang's call for "patience" regarding US-China AI dominance.
- **Product-Specific Performance and Competition:**
    - Nvidia -(struggles/to sell RTX6000D)-> GPUs
    - Nvidia -(competes/with APUs)-> AMD
    - Tencent -(breaks)-> Nvidia
    → Nvidia is reportedly struggling with sa

In [21]:
chipmaker = "AMD"
triplets = [t for d in data[chipmaker] for t in d['triplets']]

response = client.models.generate_content(
    model="gemini-2.5-flash", 
    contents=summary7day_prompt.format(
        chipmaker=chipmaker, 
        triplets=triplets
    )
)

print(response.text)

Here's an analysis of AMD's risks and opportunities over the last 7 days:

### Risks for AMD:

-   ROCm -(omits/from supported GPU list)-> Strix Halo → The omission of the new Strix Halo from ROCm's supported GPU list indicates a potential gap in AMD's AI software ecosystem for new hardware, which could limit AI development on these platforms.
-   AMD -(trails)-> Nvidia → AMD continues to trail Nvidia, highlighting a persistent competitive disadvantage, particularly in the high-growth, high-performance GPU and AI accelerator markets where Nvidia dominates.
-   AMD -(trails)-> Broadcom → Trailing Broadcom suggests competitive pressure in other key segments, potentially affecting market share or growth in areas like networking or custom silicon.
-   AMD -(has/Rowhammer DDR5)-> vulnerability → The identification of a Rowhammer DDR5 vulnerability exposes AMD to potential security risks, which could impact product reputation and require significant resources for mitigation.
-   Google -(fin

In [22]:
chipmaker = "Intel"
triplets = [t for d in data[chipmaker] for t in d['triplets']]

response = client.models.generate_content(
    model="gemini-2.5-flash", 
    contents=summary7day_prompt.format(
        chipmaker=chipmaker, 
        triplets=triplets
    )
)

print(response.text)

Here's an analysis of Intel's risks and opportunities over the last 7 days:

### Risks for Intel:

*   **Analyst Downgrade:** Citi has downgraded Intel's stock, signaling a potential decrease in investor confidence.
    *   Intel -(receives/downgrade to Sell from Neutral)-> Citi
*   **Planned Executive Transition:** The announced retirement of Chairman Wang in September 2025 introduces future leadership uncertainty, despite being planned in advance.
    *   Intel -(announces/on September 16, 2025)-> retirement
    *   Wang -(retires/as chair in September 2025)-> Intel

### Opportunities for Intel:

*   **Transformative Partnership and Investment from Nvidia:** Intel has forged a significant partnership with Nvidia, involving a $5 billion investment for a 4% stake, making Nvidia a major shareholder. This collaboration focuses on developing chips, fusing CPUs with GPUs (critical for AI/HPC), and providing strategic support in the competitive foundry landscape against rivals like TSMC. Th

In [6]:
import networkx as nx

G = nx.MultiDiGraph()

for chipmaker in data:
    for d in data[chipmaker]:
        date = d['date']
        for triplet in d['triplets']:
            subject = triplet['subject']
            relation = triplet['relation']
            object_ = triplet['object']

            verb = relation.get("verb")
            detail = relation.get("detail")

            G.add_edge(
                subject,
                object_,
                key=verb,
                verb=verb,
                detail=detail,
                date=date,
                chipmaker=chipmaker
            )